In [1]:
# %pip install transformers torch
# %pip install transformers
# %pip install ipywidgets
# %pip install numpy==1.26.4
# %pip install datasets
# %pip install scipy
# %pip install sentencepiece
# %pip install scipy==1.12
# %pip install tf-keras
# %pip install 'accelerate>=0.26.0'
# %pip install scikit-learn

In [2]:
from lib import Compressors
import numpy as np
import time


seed_value = 69
np.random.seed(seed_value)

In [3]:
def calculate(W, X, r):
    start_time = time.time()
    B = Compressors.JustSVD(W, X, r)
    svd_time = time.time() - start_time
    
    start_time = time.time()
    D = Compressors.SuperFindWeightNorm(W, X, r)
    qr_time = time.time() - start_time
    
    start_time = time.time()
    C = Compressors.MyFindWeightNorm(W, X, r)
    invert_singular_time = time.time() - start_time
    
    start_time = time.time()
    A = Compressors.AuthorFindWeightNorm(W, X, r, 10)
    author_time = time.time() - start_time
    

    norm_author = np.linalg.norm(W @ X - A @ X)
    norm_svd = np.linalg.norm(W @ X - B @ X)
    norm_invert_singular = np.linalg.norm(W @ X - C @ X)
    norm_qr = np.linalg.norm(W @ X - D @ X)


    print('Difference with reference')
    print(f"{'Author:':<20} {norm_author:12.4e} (Time: {author_time:.4f} s)")
    print(f"{'Just SVD:':<20} {norm_svd:12.4e} (Time: {svd_time:.4f} s)")
    print(f"{'Invert singular:':<20} {norm_invert_singular:12.4e} (Time: {invert_singular_time:.4f} s)")
    print(f"{'QR decomposition:':<20} {norm_qr:12.4e} (Time: {qr_time:.4f} s)")
    print('-------------------')

    # norm_author = np.linalg.norm(W - A)
    # norm_svd = np.linalg.norm(W - B)
    # norm_invert_singular = np.linalg.norm(W - C)
    # norm_qr = np.linalg.norm(W - D)

    # print('Difference with W')
    # print(f"{'Author:':<20} {norm_author:.4f}")
    # print(f"{'Just SVD:':<20} {norm_svd:.4f}")
    # print(f"{'Invert singular:':<20} {norm_invert_singular:.4f}")
    # print(f"{'QR decomposition:':<20} {norm_qr:.4f}")

def hilbert_matrix(n):
    i, j = np.indices((n, n))
    H = 1.0 / (i + j + 1)
    return H

In [4]:
W = np.random.normal(size=(768, 768))
X = np.random.normal(size=(768, 512))

r = 256
calculate(W, X, r)

Difference with reference
Author:                5.0873e+03 (Time: 0.3156 s)
Just SVD:              8.8310e+03 (Time: 0.2426 s)
Invert singular:       5.0873e+03 (Time: 0.2522 s)
QR decomposition:      5.0873e+03 (Time: 0.1752 s)
-------------------


In [5]:
W = np.random.normal(size=(3200, 3200))
X = np.random.normal(size=(3200, 1024))


r = 650
calculate(W, X, r)

Difference with reference
Author:                3.2835e+04 (Time: 6.5709 s)
Just SVD:              6.9357e+04 (Time: 5.4739 s)
Invert singular:       3.2835e+04 (Time: 1.7970 s)
QR decomposition:      3.2835e+04 (Time: 1.3900 s)
-------------------


In [6]:
W = np.random.uniform(size=(3200, 3200))
X = np.random.normal(size=(3200, 1024))

r = 1024
calculate(W, X, r)

Difference with reference
Author:                9.2623e-02 (Time: 6.4262 s)
Just SVD:              1.5563e+04 (Time: 5.3329 s)
Invert singular:       2.6296e-10 (Time: 1.7966 s)
QR decomposition:      1.5204e-10 (Time: 1.3565 s)
-------------------


In [7]:
W = np.random.uniform(size=(4000, 3200))
X = np.random.rand(3200, 512) @ np.random.rand(512, 1024)

r = 600
calculate(W, X, r)

Difference with reference
Author:                3.2757e-02 (Time: 8.2140 s)
Just SVD:              9.6642e+04 (Time: 6.7529 s)
Invert singular:       6.5124e-07 (Time: 1.7525 s)
QR decomposition:      5.8204e-07 (Time: 1.4844 s)
-------------------


In [8]:
W = np.random.uniform(size=(3200, 3200))
X = hilbert_matrix(3200)

r = 600
calculate(W, X, r)

Difference with reference
Author:                9.1554e-01 (Time: 6.3875 s)
Just SVD:              2.8502e+01 (Time: 5.2634 s)
Invert singular:       3.0149e-12 (Time: 8.5717 s)
QR decomposition:      6.1136e-12 (Time: 5.8525 s)
-------------------


In [9]:
W = np.random.uniform(size=(4000, 3200)).astype(np.float32)
X = (np.random.rand(3200, 512) @ np.random.rand(512, 1024)).astype(np.float64)

r = 600
calculate(W, X, r)

Difference with reference
Author:                3.2837e-02 (Time: 7.9205 s)
Just SVD:              9.4720e+04 (Time: 6.5729 s)
Invert singular:       5.9444e-07 (Time: 1.8343 s)
QR decomposition:      5.6130e-07 (Time: 1.4935 s)
-------------------


In [10]:
W = np.random.uniform(size=(3200, 3200)).astype(np.float32)
X = hilbert_matrix(3200).astype(np.float32)

r = 600
calculate(W, X, r)

Difference with reference
Author:                8.8916e-01 (Time: 6.2410 s)
Just SVD:              2.8543e+01 (Time: 5.1297 s)
Invert singular:       9.8949e-04 (Time: 10.1510 s)
QR decomposition:      8.0059e-04 (Time: 6.6057 s)
-------------------
